In [1]:
import pandas as pd
import requests
from forex_python.converter import CurrencyRates
import win32com.client as win32
import os 

# Send a GET request to the web page
url = "https://markets.businessinsider.com/commodities"  # Replace with the URL of the web page containing the table
response = requests.get(url)

# Read the HTML table into a list of data frames using pandas
tables = pd.read_html(response.text)

# Convert USD to INR using forex_python library
cr = CurrencyRates()
usd_to_inr = cr.get_rate('USD', 'INR')

# Create the email body
email_body = "Tables from Web Page:<br><br>"

# Process the extracted table data
for i, table in enumerate(tables):
    if 'Price' in table.columns:  # Assuming the 'Price' column is present in the table
        table['Price'] = table['Price'].astype(str)  # Convert the 'Price' column to string
        table['Price'] = table['Price'].str.replace(',', '')  # Remove comma separators in numbers
        table['Price'] = table['Price'].astype(float)  # Convert the 'Price' column to float
        table['Price_INR'] = round(table['Price'] * usd_to_inr, 2)  # Convert USD to INR
        
         # Format the 'Price' and 'Price_INR' columns in accounting format
        #table['Price'] = table['Price'].apply(lambda x: '{:,.2f}'.format(x))
        #table['Price_INR'] = table['Price_INR'].apply(lambda x: '{:,.2f}'.format(x))
        
        # Add the table content to the email body
        email_body += f"<h2>Table {i+1}</h2>"
        email_body += table.to_html(index=False) + "<br><br>"

# Create Outlook application object
outlook = win32.Dispatch("Outlook.Application")

# Create a new mail item
mail = outlook.CreateItem(0)  # 0 represents a MailItem

# Set the properties of the mail item
mail.Subject = "Real-time commodity price"
mail.HTMLBody = email_body

# Add recipients (separated by semicolon if multiple)
mail.Recipients.Add("anand@somanyceramics.com")

# Attach the tables as separate Excel files
for i, table in enumerate(tables):
    if 'Price' in table.columns:
        # Create an Excel file for each table
        filename = f"Table_{i+1}.xlsx"
        table.to_excel(filename, index=False)
        
        # Add the Excel file as an attachment
        attachment = os.path.abspath(filename)
        mail.Attachments.Add(attachment)

# Send the mail
mail.Send()

# Delete the Excel files after sending the email
for i, _ in enumerate(tables):
    if 'Price' in tables[i].columns:
        filename = f"table_{i+1}.xlsx"
        os.remove(filename)
